In [1]:
from datetime import datetime
from IPython.display import Image
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib
import numpy as np
import os
import pandas as pd
import seaborn as sns
%matplotlib inline

In [2]:
from sqlalchemy import create_engine
from sqlalchemy import Column, Integer, String, Float

from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

In [43]:
# Upload image
url = 'https://data.cityofnewyork.us/api/views/jb7j-dtam/rows.csv?accessType=DOWNLOAD'
raw_data = pd.read_csv(url)

In [3]:
raw_data.head()

NameError: name 'raw_data' is not defined

In [45]:
raw_data.dtypes

Year                        int64
Leading Cause              object
Sex                        object
Race Ethnicity             object
Deaths                     object
Death Rate                 object
Age Adjusted Death Rate    object
dtype: object

In [46]:
# identify incomplete rows
raw_data.count()

Year                       1380
Leading Cause              1380
Sex                        1380
Race Ethnicity             1380
Deaths                     1328
Death Rate                 1271
Age Adjusted Death Rate    1271
dtype: int64

In [47]:
# Drop all rows with missing information 
raw_data = raw_data.dropna(how='any')
raw_data.count()

Year                       1271
Leading Cause              1271
Sex                        1271
Race Ethnicity             1271
Deaths                     1271
Death Rate                 1271
Age Adjusted Death Rate    1271
dtype: int64

In [48]:
df = raw_data

In [49]:
df.to_csv('cleand_death.csv')

In [3]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import datetime as dt

In [4]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func,inspect

In [5]:
engine = create_engine("sqlite:///db/NY.sqlite")

In [6]:
# Create the inspector and connect it to the engine
inspector = inspect(engine)

In [7]:
inspector.get_table_names()

['Leading_Death']

In [8]:
 # Use Inspector to print the column names and types
columns = inspector.get_columns('Leading_Death')
for c in columns:
    print(c['name'], c["type"])

ID INTEGER
Year TEXT
LeadingCause TEXT
Sex TEXT
RaceEthnicity TEXT
Deaths TEXT
DeathRate TEXT
AgeAdjustedDeathRate TEXT


In [9]:
conn = engine.connect()

In [10]:
# Query All Records in the the Database
data = pd.read_sql("SELECT * FROM Leading_Death", conn)

In [11]:
data.head()

,ID,Year,LeadingCause,Sex,RaceEthnicity,Deaths,DeathRate,AgeAdjustedDeathRate
0,0,2012,All Other Causes,M,Not Stated/Unknown,78,.,.
1,1,2014,Diabetes Mellitus (E10-E14),M,Not Stated/Unknown,8,.,.
2,2,2008,"Diseases of Heart (I00-I09, I11, I13, I20-I51)",M,Black Non-Hispanic,2077,244.6,330.2
3,3,2010,All Other Causes,M,Black Non-Hispanic,1230,144.5,166.2
4,4,2014,"Accidents Except Drug Posioning (V01-X39, X43,...",F,White Non-Hispanic,169,11.9,7.4


In [12]:
# Query Single Record in the the Database
cancer = engine.execute("SELECT * FROM Leading_Death WHERE LeadingCause ='Malignant Neoplasms (Cancer: C00-C97)'")
for record in cancer:
    print(record)

(10, '2010', 'Malignant Neoplasms (Cancer: C00-C97)', 'F', 'Asian and Pacific Islander', '414', '73.6', '79.9')
(12, '2013', 'Malignant Neoplasms (Cancer: C00-C97)', 'M', 'Other Race/ Ethnicity', '32', '.', '.')
(18, '2012', 'Malignant Neoplasms (Cancer: C00-C97)', 'F', 'Other Race/ Ethnicity', '29', '.', '.')
(21, '2009', 'Malignant Neoplasms (Cancer: C00-C97)', 'M', 'Other Race/ Ethnicity', '12', '.', '.')
(22, '2007', 'Malignant Neoplasms (Cancer: C00-C97)', 'M', 'Asian and Pacific Islander', '528', '109.1', '145.9')
(32, '2009', 'Malignant Neoplasms (Cancer: C00-C97)', 'M', 'Asian and Pacific Islander', '506', '99.1', '127')
(42, '2008', 'Malignant Neoplasms (Cancer: C00-C97)', 'F', 'Asian and Pacific Islander', '344', '64.6', '73.8')
(45, '2014', 'Malignant Neoplasms (Cancer: C00-C97)', 'F', 'Other Race/ Ethnicity', '66', '.', '.')
(56, '2007', 'Malignant Neoplasms (Cancer: C00-C97)', 'M', 'Other Race/ Ethnicity', '29', '.', '.')
(93, '2011', 'Malignant Neoplasms (Cancer: C00-C97)

In [13]:
 # Reflect Database into ORM class
Base = automap_base()
Base.prepare(engine, reflect=False)

In [14]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [15]:
Base.classes.keys()

['Leading_Death']

In [16]:
Leading_Death = Base.classes.Leading_Death

In [17]:
session = Session(engine)

In [18]:
# Find those players from the USA who were born after 1989
cancer_2016 = session.query(Leading_Death).filter(Leading_Death.LeadingCause == "Malignant Neoplasms (Cancer: C00-C97)").statement
df7 = pd.read_sql_query(cancer_2016, session.bind)
df7.head()

,ID,Year,LeadingCause,Sex,RaceEthnicity,Deaths,DeathRate,AgeAdjustedDeathRate
0,10,2010,Malignant Neoplasms (Cancer: C00-C97),F,Asian and Pacific Islander,414,73.6,79.9
1,12,2013,Malignant Neoplasms (Cancer: C00-C97),M,Other Race/ Ethnicity,32,.,.
2,18,2012,Malignant Neoplasms (Cancer: C00-C97),F,Other Race/ Ethnicity,29,.,.
3,21,2009,Malignant Neoplasms (Cancer: C00-C97),M,Other Race/ Ethnicity,12,.,.
4,22,2007,Malignant Neoplasms (Cancer: C00-C97),M,Asian and Pacific Islander,528,109.1,145.9


In [35]:
result1 =session.query(Leading_Death.Year, Leading_Death.LeadingCause,Leading_Death.RaceEthnicity,func.sum(Leading_Death.Deaths).label('totaldeath')).group_by(Leading_Death.Year,Leading_Death.LeadingCause).order_by(Leading_Death.Deaths.desc()).\
        limit(10).all()
df10 = pd.DataFrame(result1, columns=['Year', 'LeadingCause', 'RaceEthnicity', 'totaldeath'])
df10.head()

,Year,LeadingCause,RaceEthnicity,totaldeath
0,2009,"Diseases of Heart (I00-I09, I11, I13, I20-I51)",Not Stated/Unknown,20084.0
1,2015,Human Immunodeficiency Virus Disease (HIV: B20...,Non-Hispanic Black,277.0
2,2016,Human Immunodeficiency Virus Disease (HIV: B20...,Non-Hispanic Black,91.0
3,2014,"Nephritis, Nephrotic Syndrome and Nephrisis (N...",Black Non-Hispanic,118.0
4,2008,Human Immunodeficiency Virus Disease (HIV: B20...,Not Stated/Unknown,927.0


In [19]:
# BONUS: Query and print the number of unique Locations
# Hints: Look into counting and grouping operations in SQLAlchemy
year = session.query(Leading_Death.Year, Leading_Death.LeadingCause,func.sum(Leading_Death.Deaths).label('totaldeath')).group_by(Leading_Death.Year,Leading_Death.LeadingCause).statement
df6 = pd.read_sql_query(year, session.bind)
df6.head(20)
# print(year)

,Year,LeadingCause,totaldeath
0,2007,"Accidents Except Drug Posioning (V01-X39, X43,...",1027.0
1,2007,All Other Causes,8142.0
2,2007,Alzheimer's Disease (G30),107.0
3,2007,Aortic Aneurysm and Dissection (I71),0.0
4,2007,"Assault (Homicide: Y87.1, X85-Y09)",381.0
5,2007,Atherosclerosis (I70),0.0
6,2007,Cerebrovascular Disease (Stroke: I60-I69),1554.0
7,2007,Certain Conditions originating in the Perinata...,0.0
8,2007,"Chronic Liver Disease and Cirrhosis (K70, K73)",125.0
9,2007,Chronic Lower Respiratory Diseases (J40-J47),1311.0


In [20]:
# Query Emojis for `emoji_char`, `emoji_id`, and `score` and save the query into results
results = session.query(Leading_Death.LeadingCause, Leading_Death.RaceEthnicity, Leading_Death.Deaths).\
    filter(Leading_Death.RaceEthnicity == 'Hispanic').\
    order_by(Leading_Death.Deaths.desc()).statement
df9 = pd.read_sql_query(results, session.bind)
df9.head()

,LeadingCause,RaceEthnicity,Deaths
0,Essential Hypertension and Renal Diseases (I10...,Hispanic,99
1,All Other Causes,Hispanic,988
2,Essential Hypertension and Renal Diseases (I10...,Hispanic,98
3,Malignant Neoplasms (Cancer: C00-C97),Hispanic,969
4,Human Immunodeficiency Virus Disease (HIV: B20...,Hispanic,96


In [ ]:
# BONUS: Use Pandas `read_sql_query` to load a query statement directly into the DataFrame
stmt = session.query(Leading_Death).\
    order_by(Leading_Death.Deaths.desc()).statement
df2 = pd.read_sql_query(stmt, session.bind)
df2.head(10)

In [22]:
# data = engine.execute("select * from Leading_Death")

# for record in data:
#     print(record.LeadingCause)

In [23]:
session.query(Leading_Death.LeadingCause).distinct().all()

[('All Other Causes'),
 ('Diabetes Mellitus (E10-E14)'),
 ('Diseases of Heart (I00-I09, I11, I13, I20-I51)'),
 ('Accidents Except Drug Posioning (V01-X39, X43, X45-X59, Y85-Y86)'),
 ('Mental and Behavioral Disorders due to Accidental Poisoning and Other Psychoactive Substance Use (F11-F16, F18-F19, X40-X42, X44)'),
 ('Influenza (Flu) and Pneumonia (J09-J18)'),
 ('Human Immunodeficiency Virus Disease (HIV: B20-B24)'),
 ('Malignant Neoplasms (Cancer: C00-C97)'),
 ('Chronic Liver Disease and Cirrhosis (K70, K73)'),
 ('Cerebrovascular Disease (Stroke: I60-I69)'),
 ('Viral Hepatitis (B15-B19)'),
 ('Insitu or Benign / Uncertain Neoplasms (D00-D48)'),
 ('Intentional Self-Harm (Suicide: X60-X84, Y87.0)'),
 ('Certain Conditions originating in the Perinatal Period (P00-P96)'),
 ('Essential Hypertension and Renal Diseases (I10, I12)'),
 ('Assault (Homicide: Y87.1, X85-Y09)'),
 ('Chronic Lower Respiratory Diseases (J40-J47)'),
 ("Parkinson's Disease (G20)"),
 ('Septicemia (A40-A41)'),
 ('Nephritis

In [24]:
 # Query for the top 10 Death rate data
session.query(Leading_Death.LeadingCause,Leading_Death.Deaths).\
        order_by(Leading_Death.Deaths.desc()).\
        limit(10).all()

[('Malignant Neoplasms (Cancer: C00-C97)', '99'),
 ('Diseases of Heart (I00-I09, I11, I13, I20-I51)', '99'),
 ('Essential Hypertension and Renal Diseases (I10, I12)', '99'),
 ('All Other Causes', '988'),
 ('Cerebrovascular Disease (Stroke: I60-I69)', '98'),
 ('Essential Hypertension and Renal Diseases (I10, I12)', '98'),
 ('Malignant Neoplasms (Cancer: C00-C97)', '969'),
 ('All Other Causes', '96'),
 ('Diabetes Mellitus (E10-E14)', '96'),
 ('Human Immunodeficiency Virus Disease (HIV: B20-B24)', '96')]

In [25]:
session.query(Leading_Death).\
    filter(Leading_Death.LeadingCause == 'Malignant Neoplasms (Cancer: C00-C97)').\
    limit(10).all()

In [26]:
print(cancer)

In [27]:
# BONUS: Use Pandas `read_sql_query` to load a query statement directly into the DataFrame
stmt = session.query(Leading_Death).\
    order_by(Leading_Death.Deaths .desc()).statement
df2 = pd.read_sql_query(stmt, session.bind)
df2.head(10)

,ID,Year,LeadingCause,Sex,RaceEthnicity,Deaths,DeathRate,AgeAdjustedDeathRate
0,285,2013,Malignant Neoplasms (Cancer: C00-C97),M,Not Stated/Unknown,99,.,.
1,723,2012,"Diseases of Heart (I00-I09, I11, I13, I20-I51)",M,Not Stated/Unknown,99,.,.
2,1103,2016,Essential Hypertension and Renal Diseases (I10...,Male,Hispanic,99,8.2,13.1
3,185,2007,All Other Causes,M,Hispanic,988,90.6,142.6
4,995,2011,Cerebrovascular Disease (Stroke: I60-I69),F,Asian and Pacific Islander,98,17.2,20.6
5,1318,2015,Essential Hypertension and Renal Diseases (I10...,Female,Hispanic,98,7.7,8.2
6,314,2007,Malignant Neoplasms (Cancer: C00-C97),F,Hispanic,969,83,100.7
7,196,2014,All Other Causes,M,Not Stated/Unknown,96,.,.
8,421,2013,Diabetes Mellitus (E10-E14),M,Asian and Pacific Islander,96,17.3,21.1
9,744,2009,Human Immunodeficiency Virus Disease (HIV: B20...,F,Hispanic,96,8,8.1


In [28]:
#df3= df2[df2['LeadingCause']=='Malignant Neoplasms (Cancer: C00-C97)']
#df3.count()

In [29]:
#df3.to_csv("cancer.csv")

In [30]:
#df3.head()

In [31]:
checking = session.query(Leading_Death).\
        order_by(Leading_Death.Deaths.desc()).statement
df8 = pd.read_sql_query(checking, session.bind)
df8.head()

,ID,Year,LeadingCause,Sex,RaceEthnicity,Deaths,DeathRate,AgeAdjustedDeathRate
0,285,2013,Malignant Neoplasms (Cancer: C00-C97),M,Not Stated/Unknown,99,.,.
1,723,2012,"Diseases of Heart (I00-I09, I11, I13, I20-I51)",M,Not Stated/Unknown,99,.,.
2,1103,2016,Essential Hypertension and Renal Diseases (I10...,Male,Hispanic,99,8.2,13.1
3,185,2007,All Other Causes,M,Hispanic,988,90.6,142.6
4,995,2011,Cerebrovascular Disease (Stroke: I60-I69),F,Asian and Pacific Islander,98,17.2,20.6


In [35]:
session.query(Leading_Death.LeadingCause,Leading_Death.Deaths,Leading_Death.RaceEthnicity,Leading_Death.Year).\
        order_by(Leading_Death.Deaths.desc()).\
        count()

1271

In [34]:
sex = session.query(Leading_Death.LeadingCause,Leading_Death.Deaths,Leading_Death.RaceEthnicity,Leading_Death.Year,Leading_Death.Sex).\
        order_by(Leading_Death.Deaths.desc()).\
        limit(1000).all()
sex_df = pd.read_sql_query(checking, session.bind)
sex_df

,ID,Year,LeadingCause,Sex,RaceEthnicity,Deaths,DeathRate,AgeAdjustedDeathRate
0,285,2013,Malignant Neoplasms (Cancer: C00-C97),M,Not Stated/Unknown,99,.,.
1,723,2012,"Diseases of Heart (I00-I09, I11, I13, I20-I51)",M,Not Stated/Unknown,99,.,.
2,1103,2016,Essential Hypertension and Renal Diseases (I10...,Male,Hispanic,99,8.2,13.1
3,185,2007,All Other Causes,M,Hispanic,988,90.6,142.6
4,995,2011,Cerebrovascular Disease (Stroke: I60-I69),F,Asian and Pacific Islander,98,17.2,20.6
5,1318,2015,Essential Hypertension and Renal Diseases (I10...,Female,Hispanic,98,7.7,8.2
6,314,2007,Malignant Neoplasms (Cancer: C00-C97),F,Hispanic,969,83,100.7
7,196,2014,All Other Causes,M,Not Stated/Unknown,96,.,.
8,421,2013,Diabetes Mellitus (E10-E14),M,Asian and Pacific Islander,96,17.3,21.1
9,744,2009,Human Immunodeficiency Virus Disease (HIV: B20...,F,Hispanic,96,8,8.1
